# How to import noise model from Qiskit  
Interestingly, different sets of Kraus operators can represent the same quantum noise process. The non-unique nature of these representations allows quantum computing libraries to use different approaches for storing and building Kraus operators to construct noise models. Here, we will first compare the construction of noise models on Qiskit and PennyLane. Then, we will learn how to convert a Qiskit noise model into an equivalent PennyLane one, allowing users to import any custom-defined or fake backend-nased noise models

## Noise models in Qiskit and PennyLane  
The noise models are in ```Qiskit-Aer``` package. Each model is represented by a ```NoiseModel``` object. Optionally, it may also have a ```ReadoutError``` that describes the classical readout errors.  

In [13]:
import numpy as np
from qiskit_aer.noise import (amplitude_damping_error, depolarizing_error,
                              pauli_error, NoiseModel)


model_qk = NoiseModel()

prob_depol = 0.2
error_gate1 = depolarizing_error(prob_depol, 1)
model_qk.add_all_qubit_quantum_error(error_gate1, ['u1', 'u2', 'u3'])

prob_bit_flip = 0.1
error_gate2 = pauli_error([('X', prob_bit_flip), ('I', 1 - prob_bit_flip)]).tensor(pauli_error([('I', 1)]))
# 위 코드는 첫번째 큐비트에 대해 prob_bit_flip 확률로 X 게이트가 적용되고, 두번째 큐비트에는 아무런 오류도 적용되지 않음
model_qk.add_all_qubit_quantum_error(error_gate2, ['cx'])



n_qubits = 3
exc_population = 0.2
prob_ampl_damp = np.random.default_rng(42).uniform(0, 0.2, n_qubits)
for q in range(n_qubits):
    error_meas = amplitude_damping_error(prob_ampl_damp[q], exc_population) 
    model_qk.add_quantum_error(error_meas, 'measure', [q]) # 해당 연산이 measure, 즉 측정할 때만 발생

print(model_qk)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u1', 'u2', 'u3']
  Instructions with noise: ['u1', 'measure', 'u2', 'u3', 'cx']
  Qubits with noise: [0, 1, 2]
  All-qubits errors: ['u1', 'u2', 'u3', 'cx']
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,))]


In constrast, the noise model in PennyLane are ```NoiseModel``` objects. It allows for a more functional construction, as we can see by recreating the above noise model as shown below.

In [14]:
import pennylane as qml

# Depolarization error for single-qubit gates
gate1_fcond = qml.noise.op_in(["U1", "U2", "U3"]) & qml.noise.wires_in(range(n_qubits))
gate1_noise = qml.noise.partial_wires(qml.DepolarizingChannel, prob_depol)

# Bit flip errors for two-qubit gate
gate2_fcond = qml.noise.op_eq("CNOT")
def gate2_noise(op, **metadata):
    qml.BitFlip(prob_bit_flip, op.wires[1])

# Readout errors for measurements
rmeas_fcond = qml.noise.meas_eq(qml.counts)
def rmeas_noise(op, **metadata):
    for wire in op.wires:
        qml.GeneralizedAmplitudeDamping(prob_ampl_damp[wire], 1 - exc_population, wire)

# Building the PennyLane noise model
model_pl = qml.NoiseModel(
    {gate1_fcond: gate1_noise, gate2_fcond: gate2_noise}, {rmeas_fcond: rmeas_noise},
)

print(model_pl)

NoiseModel({
    OpIn(['U1', 'U2', 'U3']) & WiresIn([0, 1, 2]): DepolarizingChannel(p=0.2)
    OpEq(CNOT): gate2_noise
},
meas_map = {
    MeasEq('CountsMP'): rmeas_noise
})


It is important to verify whether these noise models work the intended way.

In [15]:
# Preparing the devices
n_shots = int(2e6)
dev_pl_ideal = qml.device("default.mixed", wires=n_qubits)
dev_qk_noisy = qml.device("qiskit.aer", wires=n_qubits, noise_model=model_qk)

def GHZcircuit():
    qml.U2(0, np.pi, wires=[0])
    for wire in range(n_qubits-1):
        qml.CNOT([wire, wire + 1])
    return qml.counts(wires=range(n_qubits), all_outcomes=True)

# Preparing the circuits
pl_ideal_circ = qml.set_shots(qml.QNode(GHZcircuit, dev_pl_ideal), shots = n_shots)
pl_noisy_circ = qml.add_noise(pl_ideal_circ, noise_model=model_pl)
qk_noisy_circ = qml.set_shots(qml.QNode(GHZcircuit, dev_qk_noisy), shots = n_shots)

# Preparing the results
pl_noisy_res, qk_noisy_res = pl_noisy_circ(), qk_noisy_circ()

In [16]:
pl_probs = np.array(list(pl_noisy_res.values())) / n_shots
qk_probs = np.array(list(qk_noisy_res.values())) / n_shots

print("PennyLane Results: ", np.round(pl_probs, 3))
print("Qiskit Results:    ", np.round(qk_probs, 3))
print("Are results equal? ", np.allclose(pl_probs, qk_probs, atol=1e-2))

PennyLane Results:  [0.385 0.057 0.028 0.076 0.056 0.028 0.082 0.287]
Qiskit Results:     [0.386 0.057 0.028 0.076 0.056 0.028 0.082 0.287]
Are results equal?  True


## Importing Qiskit noise models  
PennyLane provides the ```from_qiskit_noise()``` function to easily convert a Qiskit noise model into an equivalent PennyLane noise model.

In [22]:
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime.fake_provider import FakeFez

# backend = GenericBackendV2(num_qubits=2, seed=42)
backend = FakeFez()
qk_noise_model = NoiseModel.from_backend(backend)
print(qk_noise_model)

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['cz', 'x', 'measure', 'id', 'sx', 'reset']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('cz', (0, 1)), ('cz', (1, 0)), ('cz', (1, 2)), ('cz', (2, 1)), ('cz', (2, 3)), ('cz', 

In [23]:
pl_noise_model = qml.from_qiskit_noise(qk_noise_model)
print(pl_noise_model)

NoiseModel({
    OpIn(['CZ']) & WiresIn([0, 1]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([1, 0]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([1, 2]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([2, 1]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([2, 3]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([3, 2]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([3, 4]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([3, 16]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([4, 3]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([4, 5]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([5, 4]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([5, 6]): QubitChannel(num_kraus=16, num_wires=2)
    OpIn(['CZ']) & WiresIn([6, 5]): QubitChannel(num_kraus=16, num_wires=2

/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/pennylane_qiskit/noise_models.py:134: UserWarning: Readout errors are not supported currently and will be skipped.
  warn("Readout errors are not supported currently and will be skipped.")


In [25]:
pl_ideal_circ

<QNode: device='<default.mixed device (wires=3) at 0x7f4aa31bfa50>', interface='auto', diff_method='best', shots='Shots(total=2000000)'>

In [29]:
pl_ideal_circ = qml.set_shots(qml.QNode(GHZcircuit, dev_pl_ideal), shots = n_shots)
pl_noisy_circ = qml.add_noise(pl_ideal_circ, noise_model=pl_noise_model)

pl_ideal_= pl_ideal_circ()
pl_noisy_ = pl_noisy_circ()

pl_ideal = np.array(list(pl_ideal_.values())) / n_shots
pl_noisy = np.array(list(pl_noisy_.values())) / n_shots

print("PennyLane Results: ", np.round(pl_ideal, 3))
print("Qiskit Results:    ", np.round(pl_noisy, 3))
print("Are results equal? ", np.allclose(pl_ideal, pl_noisy, atol=1e-2))

ValueError: OpIn does not support arithmetic operations that cannot be converted to a linear combination